In [1]:
from langchain_openai import ChatOpenAI, OpenAIEmbeddings
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder, FewShotPromptTemplate, PromptTemplate
from langchain_core.output_parsers import StrOutputParser, JsonOutputParser, PydanticOutputParser
from langchain_core.runnables import RunnablePassthrough, RunnableParallel
from langchain_core.messages import HumanMessage
from langchain_chroma import Chroma
from langchain_core.example_selectors import SemanticSimilarityExampleSelector

c:\Coding\Local\exercise_book_ver2\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
prompt = ChatPromptTemplate.from_template(
    '주제 {topic}에 대해 짧은 설명을 해주세요.'
)
output_parser = StrOutputParser()
model = ChatOpenAI(model='gpt-5-nano')

In [3]:
chain = (
    {'topic':RunnablePassthrough()}
    | prompt
    | model
    | output_parser
)

print(chain.invoke('더블딥'))

더블딥은 분야에 따라 다른 의미로 쓰입니다. 두 가지 흔한 해석을 간단히 소개합니다.

- Double DQN(강화학습): Q-학습의 과대추정을 줄이기 위해 정책 선택과 가치 평가를 서로 다른 두 네트워크로 분리하는 기법입니다. 이렇게 하면 에이전트의 학습이 더 안정적으로 이뤄집니다.
- 이중침체(경제학): 경기가 한 차례 회복을 보이다가 다시 침체에 빠지는 현상을 말합니다. 보통 "이중 침체"로도 불립니다.

원하시는 분야를 알려주시면 그에 맞춰 더 간단히 설명해 드릴게요.


In [4]:
prompt = ChatPromptTemplate.from_template(
    '주제 {topic}에 대해 짧은 설명을 해주세요.'
)
output_parser = StrOutputParser()
model = ChatOpenAI(model='gpt-5-nano', 
                   #temperature=1,
                   #max_completion_tokens=1000
                   )

In [5]:
chain = (
    {'topic':RunnablePassthrough()}
    | prompt
    | model
    | output_parser
)

for chunk in chain.stream('더블딥'):
    print(chunk, end='', flush=True)

혹시 강화학습의 용어인 "더블 DQN(Double DQN)"를 말씀하시는 건가요? 아니면 다른 뜻의 "더블딥"을 의미하시는지 확인이 필요합니다. 만약 Double DQN이라면 짧은 설명은 다음과 같습니다:

- 더블 DQN은 Q-값의 과대추정을 줄이기 위해 정책 선택과 가치 평가를 서로 다른 두 네트워크로 분리해 학습합니다. 하나의 네트워크가 행동을 선택하고, 다른 네트워크가 그 선택의 가치를 평가해 타깃 값을 업데이트합니다.

다른 뜻이라면 어떤 주제를 원하시는지 알려주시면 짧게 설명해 드리겠습니다.

In [6]:
prompt = ChatPromptTemplate.from_template(
    '주제 {topic}에 대해 짧은 설명을 해주세요.'
)
output_parser = StrOutputParser()
model = ChatOpenAI(model='gpt-5-nano', 
                   #temperature=1,
                   #max_completion_tokens=1000
                   )

In [7]:
chain = (
    {'topic':RunnablePassthrough()}
    | prompt
    | model
    | output_parser
)

print(chain.batch([{'topic':'더블딥'}, {'topic':'인플레이션'}])[0])
print(chain.batch([{'topic':'더블딥'}, {'topic':'인플레이션'}])[1])

더블딥은 한 번 침체를 벗어나려는 듯 보이다가 다시 경기 하강에 빠지는 현상을 말합니다. 이중 침체라고도 부릅니다.

- 보통 실질 GDP가 두 분기 연속 감소하거나, 짧은 회복 뒤에 다시 약화되며 재침체가 나타납니다.
- 원인으로 수요 부진, 금융 여건 악화, 정책 반응의 지연 등이 작용하고, 고용과 투자에도 큰 타격이 있습니다.
다음은 인플레이션에 대한 짧은 설명입니다.

- 정의: 전반적인 물가가 상승해 같은 돈으로 살 수 있는 상품·서비스의 양이 줄어드는 현상.
- 측정: 소비자물가상승률(CPI) 등으로 연간 물가 상승률을 나타냄.
- 주된 원인: 수요 증가에 따른 수요주도 인플레이션, 생산비 상승·원자재 가격 상승 등 비용주도 인플레이션, 물가 기대의 상승 등.
- 영향: 실질 구매력 감소, 저축 가치 변화, 금리와 고용에 영향.
- 정책: 중앙은행의 물가안정 목표와 금리 정책으로 수요를 관리하려는 노력.


In [8]:
analysis_prompt = ChatPromptTemplate.from_template('이 대답을 영어로 번역해주세요: {answer}')
composed_chain = (
    {'answer':chain}
    | analysis_prompt
    | model
    | output_parser
)
print(composed_chain.invoke({'topic':'더블딥'}))

A Double Dip is the phenomenon where the economy, after experiencing one recession, briefly recovers and then falls back into recession. The recovery does not rise above the previous trough, which is a characteristic of the pattern, and it is often described as a "double bottom" pattern. Causes include various factors such as policy lags, weak demand, and deteriorating financial conditions.


In [9]:
prompt = ChatPromptTemplate.from_template('{topic}에 대해서 짧은 설명을 해주세요.')
chain = (
    {'topic':RunnablePassthrough()}
    | prompt
    | model
    | output_parser
)
analysis_prompt = ChatPromptTemplate.from_template('이 대답을 영어로 번역해주세요.:{answer}')
composed_chain_with_lambda = (
    chain
    | (lambda input: {'answer':input})
    | analysis_prompt
    | model
    | output_parser
)
print(composed_chain_with_lambda.invoke({'topic':'더블딥'}))

A double dip refers to an economic phenomenon in which the economy experiences a period of recovery, only to slip back into a recession. In other words, after a rebound GDP falls again and employment and other indicators deteriorate, a pattern sometimes likened to a “W-shaped recovery.” The main causes include weak demand, the aftereffects of financial shocks, and lags in the effects of policy.


In [10]:
composed_chain_with_pipe = chain.pipe(
    lambda input: {'answer':input}, analysis_prompt, model, output_parser
)
print(composed_chain_with_pipe.invoke({'topic':'더블딥'}))

Double dip is primarily an economic term that refers to a phenomenon where, after one recession ends, there is a brief recovery before slipping back into recession. In other words, it describes a situation in which two recessions occur in succession. If you’d like, I can also explain its meaning in other fields.


In [11]:
kor_chain = (
    ChatPromptTemplate.from_template('{topic}에 대한 짧은 설명을 해주세요.')
    | model
    | output_parser
)
eng_chain = (
    ChatPromptTemplate.from_template('{topic}에 대해 짧게 영어로 설명을 해주세요.')
    | model
    | output_parser
)
parallel_chain = RunnableParallel(kor=kor_chain, eng=eng_chain)
result = parallel_chain.invoke({'topic':'더블딥'})
print(result['kor'])
print(result['eng'])

더블딥(DQN)의 한 버전으로, Q-학습의 과대추정을 줄이기 위해 학습에서 행동 선택과 평가를 서로 다른 네트워크가 담당하도록 하는 방법입니다. 
구현 방식은: 온라인 네트워크가 s'에서 최적 행동 a'를 선택하고, 타깃 네트워크가 그 행동의 값을 평가해 y = r + γ Q(s', a'; θ^-) 를 목표로 학습합니다. 타깃 네트워크는 주기적으로 또는 소프트 업데이트로 가중치를 반영합니다. 
이렇게 두 네트워크를 분리하면 학습의 안정성이 높아지고 성능이 향상되는 경향이 있습니다.
Double DQN is an extension of DQN that reduces overestimation in Q-learning.

- It uses two neural networks: an online network for selecting actions and a separate target network for evaluating them.
- During update, actions for the next state are chosen by the online network, but their values are estimated by the target network.
- The target value is y = r + gamma * Q_target(next_state, argmax_a Q_online(next_state, a)).
- The target network is updated slowly to stabilize learning.

This decoupling makes learning more stable and often improves performance.


In [12]:
prompt_template = ChatPromptTemplate.from_messages(
    [('system', '당신은 유능한 금융 조언가입니다.'),
     ('user', '주제 {topic}에 대하여 금융 관련 조언을 해주세요.')]
)
chain = (
    {'topic':RunnablePassthrough()}
    | prompt_template
    | model
    | output_parser
)
print(chain.invoke({'topic':'더블딥'}))

주제: 더블딥(이중 침체)에 대한 금융 관련 조언

더블딥은 경제가 일시적으로 회복 국면에 접어든 뒤 다시 침체로 전환되는 시나리오를 말합니다. 원인으로는 성장 모멘텀의 약화, 실물 수요의 급감, 고물가나 금리 인상으로 인한 차입 비용 증가, 정책 효과의 한계 등이 있습니다.

주요 신호(모니터링 포인트)
- GDP 성장률의 재하강 또는 기대치 하회
- 고용 지표의 악화 또는 고용회복의 이질성
- 제조업/서비스업 PMI의 재하강
- 소비심리/소비지출 부진
- 금리 인상 속도 조절 여부 또는 금융시장 불안
- 가계부채 부담 증가와 재정/통화 정책의 변화

더블딥에 대비한 투자 원칙
- 위험 관리 우선: 원금을 지키는 것이 우선이 될 수 있습니다. 과도한 위험 자산 비중은 recession 시 손실을 키울 수 있습니다.
- 분산과 다변화: 국내외 주식, 채권, 현금성 자산, 금 등 대체자산까지 폭넓은 분산이 필요합니다.
- 질 좋은 자산 위주: 안정적 현금흐름과 재무구조가 탄탄한 기업의 주식, 등급이 높은 채권 중심으로 구성하는 것이 방어적입니다.
- 현금흐름 관리: 비상금 확보와 단기 필요자금의 원천을 구분해 운용합니다.
- 긴 호흡의 자산배분: 단발성 타이밍보다는 시나리오 기반의 재조정이 필요합니다.

실전 전략(일반적인 가이드)
1) 유동성 확보
- 비상금으로 6-12개월의 생활비에 해당하는 자금을 현금성 자산으로 보유.
- 단기 국채/머니마켓펀드처럼 쉽게 현금화 가능한 자산 비중을 늘려 변동성 대응력을 높이세요.

2) 방어적 주식과 질 좋은 고배당/필수소비재 비중 확대
- 헬스케어, 필수소비재, 유틸리티 등 경기 민감도 낮은 섹터의 비중을 약간 확대.
- 변동성이 큰 성장주보단 품질주 중심으로 포트폴리오를 안정화합니다.

3) 질 높은 채권 중심의 고정수익 강화
- 단기에서 중기 만기 투자등급 채권, 국채 중심으로 구성해 이자소득과 원금보전을 강화.
- 금리하락 국면에 대비해 듀레이션 관리도 병행(필요 시 적정한 듀레이션 조정).

4) 인플레이션과 금리

In [13]:
prompt_template = ChatPromptTemplate.from_messages(
    [('system', '당신은 유능한 금융 조언가입니다.'),
     ('user', '주제 {topic}에 대하여 금융 관련 조언을 해주세요.')]
)
chain = (
    #{'topic':RunnablePassthrough()}
    prompt_template
    | model
    | output_parser
)
print(chain.invoke({'topic':'더블딥'}))

다음은 주제인 “더블딥”(이중 침체)에 대해 일반적으로 참고할 수 있는 금융 관련 조언입니다. 개인 상황에 맞춘 구체적 투자 조언은 신중히 전문가와 상의하시길 권합니다.

1) 더블딛이란 무엇인가
- 정의: 경제가 한 차례 심각한 침체를 겪은 뒤 짧은 반등을 보이다가 다시 침체로 내려앉는 상황을 말합니다. 흔히 V자 회복이 아닌 이중 하강 형태로 나타납니다.
- 원인 요인: 수요 부진의 재발, 고부채 부담, 공급 충격의 재현, 정책의 미스매치 등.
- 금융시장에의 영향: 불확실성과 변동성 증가, 위험자산의 하락 재개 가능성, 채권과 현금의 상대적 가치 변화.

2) 더블딥 시기에 주의해야 할 지표/징후
- 실물경제: 2분기 이상 GDP가 연속 감소하거나 고용이 급격히 악화될 때.
- 금융시장: 주가의 추가 급락 우려, 기업 실적 부진, 신용여건 악화.
- 정책환경: 중앙은행의 금리 방향성 전환 신호, 재정정책의 규모와 시기가 불확실해지는 경우.

3) 더블딥 가능성에 대비하는 일반적 전략
- 현금 흐름과 재무 건전성 강화
  - 비상금 마련: 생활비와 고정지출을 최소 6~12개월치 수준으로 확보하는 것이 일반적으로 권장됩니다.
  - 고금리 부채 우선 상환: 변동금리 대출이나 신용카드 부채 등 비용이 큰 부채를 먼저 정리합니다.
- 포트폴리오 방어력 강화
  - 자산배분의 기본에 충실: 주식과 채권의 균형, 위험 허용도에 맞춘 분산.
  - 방어적 주식 선호: 품질이 좋고 현금 흐름이 탄탄한 기업의 주식 비중 유지 또는 확대(필수소비재, 헬스케어, 유틸리티 등).
  - 고품질 채권 위주: 신용등급이 높은 단기/중기 채권으로 구성하고, 필요시 인플레이션 연동채(TIPS 등)도 고려.
- 시장 변동성 대비 다각화
  - 지역 다각화: 국내 시장에 의존하기보다 선진국/신흥국의 비중 균형 조정.
  - 자산 클래스 다각화: 주식 외에 현금성 자산, 채권, 일부 대체자산(헤지펀드 스타일의 저위험 전략 등)을 혼합.
- 투자 접근 방식
  - 달러 비용 평균화(Doll

In [14]:
prompt_template = ChatPromptTemplate.from_messages(
    [('system', '당신은 유능한 금융 조언가입니다.'),
     MessagesPlaceholder('msgs')]
)
chain = (
    #{'topic':RunnablePassthrough()}
    prompt_template
    | model
    | output_parser
)
print(chain.invoke({'msgs':[HumanMessage(content='안녕하세요. 더블딥에 대해 알려주세요.')]}))

반갑습니다. 더블딥은 주로 거시경제 용어로 쓰이며, “두 번째 경기 침체가 짧은 기간의 반등 뒤에 다시 찾아오는 현상”을 말합니다. 자세히 정리해 드릴게요.

1) 더블딥의 정의
- 기본 아이디어: 한 번의 경기 침체가 끝나고 회복 국면이 시작된 듯 보였는데, 그 회복이 이어지지 못하고 다시 경제가 위축되는 경우를 말합니다.
- 공식 판정과의 차이: 각 나라의 경기 사이클 판단기관은 다르게 정의합니다. 예를 들어 미국은 NBER가 경기 침체 여부를 판단하고, 두 번의 연속 마이너스 성장만으로 자동으로 규정하지 않는 경우가 많습니다. 따라서 “더블딥”은 주로 언론이나 시장 참가자 사이의 용어로도 쓰이고, 실질로는 두 번의 공식 침체 여부가 나와야 확정되기도 합니다.

2) 왜 생길 수 있나요? (주요 원인)
- 가계 부채 부담과 소비 약화로 인한 수요 급감
- 기업 투자 감소와 생산 차질로 인한 생산성 악화
- 정책 불확실성(금리 정책, 재정정책 방향의 변화)으로 인한 지연적 소비/투자
- 외부 충격(전쟁, 원자재 가격 급등락, 팬데믹 이후 회복의 바람직한 속도 저하)
- 금융시장 충격과 신용 여건 악화로 인한 재투자 위축

3) 흔히 보이는 징후/지표
- 분기 GDP가 연속해서 마이너스인 기간이 짧게 끝난 뒤 다시 마이너스로 돌아감(공식 판단과 시점 차이 가능)
- 실업률 상승 재개, 소비지표 약세(소매판매, 소비심리 악화)
- 제조업 PMI나 산업생산 지표의 재하락
- 인플레이션/금리 흐름이 정책을 어렵게 하거나, 정책 변화에 따른 기대 변화가 불확실해짐
- 금융시장 변동성 증가와 신용 여건 악화

4) 역사적 맥락
- 더블딥은 비교적 드물고, 특정 지역에서의 경기 사이클 맥락에서 언급되는 경우가 많습니다. 예를 들어 global 금융위기 이후 회복 국면에서 “다시 침체로 돌아설 수 있다”는 우려가 제기되기도 했지만, 공식적으로 두 번의 연속 마이너스 GDP로 확정되지는 않기도 했습니다. 따라서 상황마다 다르고, 데이터가 확정적으로 나오기 전까지는 단정하기 

In [15]:
prompt_template = ChatPromptTemplate.from_messages(
    [('system', '당신은 유능한 금융 조언가입니다.'),
     ('placeholder', '{msgs}')]
)
chain = (
    #{'topic':RunnablePassthrough()}
    prompt_template
    | model
    | output_parser
)
print(chain.invoke({'msgs':[HumanMessage(content='안녕하세요. 더블딥에 대해 알려주세요.')]}))

안녕하세요! 더블딥은 보통 경제가 한 차례 침체를 겪은 뒤 잠시 회복했다가 다시 침체로 들어서는 현상을 말합니다. 간단하게 정리해 드리면:

1) 정의
- 더블딥은 두 번의 경기 침체가 연이어 발생하는 상황을 가리킵니다.
- 일반적인 간이 기준으로는 실질 GDP가 두 분기 연속 감소하는 구간이 서로 떨어져 있다가 다시 침체로 이어지는 경우를 말하곤 합니다. 다만 공식 침체 여부는 각 나라의 경기연구기관(NBER 등)에서 판단합니다.

2) 왜 생길까(발생 원인)
- 수요 급감: 소비와 기업 설비 투자 등이 급락합니다.
- 금융 제약: 신용 경색, 이자율 급등 혹은 시장 불안이 지속될 때 회복이 흔들립니다.
- 정책 효과의 이중성: 경기부양책의 효과가 한동안만 지속되고, 이후 추가 지원이 부족하면 재침체로 이어질 수 있습니다.
- 외부 shock: 에너지 가격 급등/급락, 글로벌 공급망 붕괴 등 외부 충격이 재차 발생하면 회복이 흔들립니다.

3) 신호와 지표
- GDP 성장률이 두 분기 연속 마이너스로 접하면 분위기가 악화되기 시작합니다.
- 실업률 상승, 소비지출 약화, 제조업/서비스업 PMI 악화 등의 선행 지표가 약세를 보입니다.
- 금융시장 변수로는 주가 하락과 함께 채권 수익률 곡선의 변화, 신용스프레드 확대 등이 동반될 수 있습니다.
- 공식적으로는 각 국의 경기연구기관이 침체의 시작과 종료를 판단합니다.

4) 역사적 사례(참고)
- 1980년대 미국은 두 차례의 침체가 짧은 회복을 사이에 두고 이어진 사례가 언급되곤 합니다(더블딥에 대한 대표적 예로 자주 인용).
- 2008년 금융위기 이후 글로벌 회복 과정에서 더블딥 우려가 제기되었지만, 공식적으로 두 차례의 침체가 확정되지는 않는 경우가 많았습니다. 즉, “더블딥 우려”와 실제 침체 여부는 다를 수 있습니다.

5) 투자자 관점에서의 대응 포인트
- 방어적 주식 및 채권 구성: 경기 둔화 가능성에 대비해 헬스케어/필수소비재 등 방어적 섹터의 비중을 고려하고, 고품질의 단기/중기 채권으로 위험

In [16]:
examples = [
    {
        "question": "주식 투자와 예금 중 어느 것이 더 수익률이 높은가?",
        "answer": """
후속 질문이 필요한가요: 네.
후속 질문: 주식 투자의 평균 수익률은 얼마인가요?
중간 답변: 주식 투자의 평균 수익률은 연 7%입니다.
후속 질문: 예금의 평균 이자율은 얼마인가요?
중간 답변: 예금의 평균 이자율은 연 1%입니다.
따라서 최종 답변은: 주식 투자
""",
    } ,
    {
        "question": "부동산과 채권 중 어느 것이 더 안정적인 투자처인가?",
        "answer": """
후속 질문이 필요한가요: 네.
후속 질문: 부동산 투자의 위험도는 어느 정도인가요?
중간 답변: 부동산 투자의 위험도는 중간 수준입니다.
후속 질문: 채권의 위험도는 어느 정도인가요?
중간 답변: 채권의 위험도는 낮은 편입니다.
따라서 최종 답변은: 채권
""",
    },
]

example_prompt = PromptTemplate.from_template('질문: {question}\n답변: {answer}')
print(example_prompt.invoke(examples[0]).to_string())

질문: 주식 투자와 예금 중 어느 것이 더 수익률이 높은가?
답변: 
후속 질문이 필요한가요: 네.
후속 질문: 주식 투자의 평균 수익률은 얼마인가요?
중간 답변: 주식 투자의 평균 수익률은 연 7%입니다.
후속 질문: 예금의 평균 이자율은 얼마인가요?
중간 답변: 예금의 평균 이자율은 연 1%입니다.
따라서 최종 답변은: 주식 투자



In [17]:
prompt = FewShotPromptTemplate(examples=examples, example_prompt=example_prompt, suffix='질문: {input}', input_variables=['input'])
print(prompt.invoke({'input':'부동산 투자의 장점은 무엇인가?'}).to_string())

질문: 주식 투자와 예금 중 어느 것이 더 수익률이 높은가?
답변: 
후속 질문이 필요한가요: 네.
후속 질문: 주식 투자의 평균 수익률은 얼마인가요?
중간 답변: 주식 투자의 평균 수익률은 연 7%입니다.
후속 질문: 예금의 평균 이자율은 얼마인가요?
중간 답변: 예금의 평균 이자율은 연 1%입니다.
따라서 최종 답변은: 주식 투자


질문: 부동산과 채권 중 어느 것이 더 안정적인 투자처인가?
답변: 
후속 질문이 필요한가요: 네.
후속 질문: 부동산 투자의 위험도는 어느 정도인가요?
중간 답변: 부동산 투자의 위험도는 중간 수준입니다.
후속 질문: 채권의 위험도는 어느 정도인가요?
중간 답변: 채권의 위험도는 낮은 편입니다.
따라서 최종 답변은: 채권


질문: 부동산 투자의 장점은 무엇인가?


In [18]:
example_selector = SemanticSimilarityExampleSelector.from_examples(
    examples,
    OpenAIEmbeddings(),
    Chroma,
    k=1
)

In [19]:
question = '부동산 투자의 장점은 무엇인가?'
print(f'입력질문: {question}')
selected_examples = example_selector.select_examples({'question':question})
for example in selected_examples:
    print('\n')
    print('# 입력과 가장 유사한 예제:')
    for k, v in reversed(example.items()):
        print(f'{k}: {v}')

입력질문: 부동산 투자의 장점은 무엇인가?


# 입력과 가장 유사한 예제:
answer: 
후속 질문이 필요한가요: 네.
후속 질문: 부동산 투자의 위험도는 어느 정도인가요?
중간 답변: 부동산 투자의 위험도는 중간 수준입니다.
후속 질문: 채권의 위험도는 어느 정도인가요?
중간 답변: 채권의 위험도는 낮은 편입니다.
따라서 최종 답변은: 채권

question: 부동산과 채권 중 어느 것이 더 안정적인 투자처인가?


In [20]:
example_prompt = PromptTemplate(
    input_variables=['question', 'answer'],
    template='질문: {question}\n답변: {answer}'
)

prompt = FewShotPromptTemplate(
    example_selector=example_selector,
    example_prompt=example_prompt,
    prefix='다음은 금융 관련 질문과 답변의 예입니다.',
    suffix='질문: {question}\n답변:',
    input_variables=['input']
)

In [21]:
model = ChatOpenAI(model='gpt-5-nano')
chain = (
    prompt
    | model
    #| output_parser
)
response = chain.invoke({'question': '부동산 투자의 장점은 무엇인가?'})
print(response.content)

답변:
- 임대 수입을 통한 정기적 현금흐름: 임대를 통한 꾸준한 수입으로 현금 흐름이 안정될 수 있습니다.
- 자본 상승 가능성: 부동산 가치가 상승하면 매매 시 시세 차익을 얻을 수 있습니다.
- 레버리지 활용: 대출을 이용해 초기 자본 대비 더 큰 규모의 투자가 가능해 수익률을 확대할 수 있습니다.
- 인플레이션 헤지: 물가 상승 시 임대료와 자산가치가 함께 상승하는 경향이 있어 인플레이션에 상대적으로 강한 편입니다.
- 포트폴리오 다변화 및 위험 분산: 주식·채권과의 상관관계가 낮아 포트폴리오의 전반적 위험을 분산하는 데 도움이 됩니다.
- 세제 혜택 및 감가상각: 지역에 따라 세제 혜택이나 감가상각 혜택을 활용해 순이익이 개선될 수 있습니다.
- 관리 및 가치 창출 가능성: 리모델링이나 관리 개선을 통해 자산 가치를 직접 높일 수 있습니다.
- 실물 자산의 제어권: 물리적 자산을 보유하고 직접 관리하는 선택이 가능합니다.

참고: 지역·시장 상황에 따라 장점의 크기와 실현 가능성은 달라지며, 유동성은 일반적으로 채권/주식에 비해 낮은 편입니다.


In [22]:
parser = JsonOutputParser()
instructions = parser.get_format_instructions()
print(instructions)

Return a JSON object.


In [23]:
ai_response = '{"이름":"김철수", "나이":30}'
parsed_response = parser.parse(ai_response)
print(parsed_response)

{'이름': '김철수', '나이': 30}
